In [1]:
import tempfile
import os
from azureml.core import Workspace, Dataset, Datastore
from azureml.data import dataset_factory, datapath
from IPython.display import display
from DataProcessing import combine_notes, clean, text_cleaning, replace_abbrevs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep='\n')

# previously registered as Datastores
pldev_dw_ds = Datastore.get(workspace, 'plddev_dw_sql_db')
hanoverlake_ds = Datastore.get(workspace, 'hanoverlake_adls_gen1')
plaadl_ds = Datastore.get(workspace, 'plaadl')

plds_ml
machine_learning
eastus2
51bbe03c-c84f-488f-9e90-446bc59e683e


In [3]:
query = datapath.DataPath(pldev_dw_ds, "SELECT * FROM SANDBOX.clm_notes_WW WHERE (WATER_WEATHER = 1 OR WATER_NONWEATHER = 1) AND masked_notetext NOT LIKE 'Restoring claim from M2A accelerator' AND author != 'HCS Converted'")
tabular = Dataset.Tabular.from_sql_query(query, query_timeout=10)
test_df = tabular.to_pandas_dataframe()

Resolving access token for scope "https://database.windows.net//.default" using identity of type "MANAGED".
Getting data access token with Assigned Identity (client_id=clientid) and endpoint type based on configuration


In [ ]:
test_df.head()

In [4]:
text_cleaning(test_df, 'masked_notetext')

0          coverage analysis ho homeowners special form l...
1          inspection investigation present at inspection...
2                                       approve reassignment
3          water mitigation vendor name service master by...
4          policy in force at time of loss a dwelling b o...
                                 ...                        
1102690    pipe break invoice on file recon a rcv deducti...
1102691    received hoa insurance quote for the repairs w...
1102692    obc to insd lvm requesting call back to scope ...
1102693    using short template due to cat claim volume v...
1102694    call from insd sp came in and drilled holes in...
Name: cleaned_text, Length: 1102695, dtype: object

In [ ]:
test_df.head()

In [9]:
import pandas as pd
def replace_abbrevs(df:pd.DataFrame, text_field:str, abbreviations:dict={}, out_col_name:str="prepped_text"):
    """
    prepare text field by stripping extra spaces, and if desired replace abbrevations

    Params:
    df: pandas data frame with text field
    text_field: column name of text field to be altered
    abbreviations: dictionary of possible abbreviations to be converted to full text, default: {}
    out_col_name: column name of output text column, default: prepped_text

    Returns:
    df[prepped_text] --> column that contains abbreviation replacements
    """

    if abbreviations !={}:
        abbreviations = {r'(?i)\W{} '.format(key): " " + val + " " for key, val in abbreviations.items()}
        df['prepped_text'] = df[text_field].replace(abbreviations, regex=True).str.strip()
        #df.loc[:, out_col_name] = df.loc[:, out_col_name].replace(abbreviations, 
                                             # regex=True).str.strip()
    return df['prepped_text']

In [10]:
replace_abbrevs(test_df, 'cleaned_text', {'insd':'insured', 'ppl':'people', 'dmgd':'damaged', 'cvg':'coverage'})

0          coverage analysis ho homeowners special form l...
1          inspection investigation present at inspection...
2                                       approve reassignment
3          water mitigation vendor name service master by...
4          policy in force at time of loss a dwelling b o...
                                 ...                        
1102690    pipe break invoice on file recon a rcv deducti...
1102691    received hoa insurance quote for the repairs w...
1102692    obc to insured lvm requesting call back to sco...
1102693    using short template due to cat claim volume v...
1102694    call from insured sp came in and drilled holes...
Name: prepped_text, Length: 1102695, dtype: object

In [ ]:
test_df